In [1]:
import sys
import os

# Set project root (assumes this notebook is in ./notebooks)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

if project_root not in sys.path:
    sys.path.insert(0, project_root)
from src.core import core

dlls = core.generate_dlls(use_contract = True)
dll = dlls['V18']
contract = dlls['Siemens.Engineering.Contract']

import clr  # noqa: E402
from System.IO import DirectoryInfo, FileInfo  # noqa: E402

clr.AddReference(dll.as_posix())
clr.AddReference(contract.as_posix())

import Siemens.Engineering as SE  # noqa: E402
import src.modules.Portals as Portals


imports = Portals.Imports(SE, DirectoryInfo, FileInfo)




In [2]:
from pathlib import Path, PurePosixPath
import json

from src.schemas import configuration

BASE_DIR = Path(".").absolute().parent
smc = BASE_DIR / "tests" / "configs" / "smc.json"

CONFIG = None
with open(smc) as file:
    CONFIG = configuration.validate(json.load(file))


In [3]:
PID = 12848

In [4]:
settings = {"enable_ui": True, "connection_method": {'mode': 'attach', "process_id": PID}}
TIA = Portals.connect(imports, CONFIG, settings)


[2025-08-06 10:22:22] INFO [src.modules.Portals:36] - Attached TIA Portal Openness (12848) WithUserInterface at 8/6/2025 2:22:22 AM


In [5]:
import src.modules.Devices as Devices
device = TIA.Projects[0].Devices[0]
plcsoftware = Devices.get_plc_software(imports, device)
plcsoftware

[2025-08-06 10:22:22] DEBUG [src.modules.Devices:44] - Accessing a PlcSoftware from Device Item Rack_0
[2025-08-06 10:22:22] DEBUG [src.modules.Devices:51] - No Software Container for Device Item Rack_0
[2025-08-06 10:22:22] DEBUG [src.modules.Devices:44] - Accessing a PlcSoftware from Device Item PLC_1
[2025-08-06 10:22:22] DEBUG [src.modules.Devices:61] - Found PlcSoftware for Device Item PLC_1


In [7]:
import src.modules.PlcBlocks as PlcBlocks

# make sure to compile the software first
OB = PlcBlocks.find(plcsoftware, PurePosixPath("/"), "Main")
PlcBlocks.export_xml(imports, OB)

[2025-08-06 10:25:10] INFO [root:19] - Started export of PlcBlock Main XML
[2025-08-06 10:25:20] DEBUG [root:32] - Extracted XML: <?xml version="1.0" encoding="utf-8"?>
<Document>
  <Engineering version="V18" />
  <DocumentInfo>
    <Created>2025-08-06T02:25:19.8415072Z</Created>
    <ExportSetting>None</ExportSetting>
    <InstalledProducts>
      <Product>
        <DisplayName>Totally Integrated Automation Portal</DisplayName>
        <DisplayVersion>V18</DisplayVersion>
      </Product>
      <OptionPackage>
        <DisplayName>TIA Portal Openness</DisplayName>
        <DisplayVersion>V18</DisplayVersion>
      </OptionPackage>
      <OptionPackage>
        <DisplayName>TIA Portal Version Control Interface</DisplayName>
        <DisplayVersion>V18</DisplayVersion>
      </OptionPackage>
      <Product>
        <DisplayName>STEP 7 Professional</DisplayName>
        <DisplayVersion>V18</DisplayVersion>
      </Product>
      <OptionPackage>
        <DisplayName>STEP 7 Safety</Display

'<?xml version="1.0" encoding="utf-8"?>\n<Document>\n  <Engineering version="V18" />\n  <DocumentInfo>\n    <Created>2025-08-06T02:25:19.8415072Z</Created>\n    <ExportSetting>None</ExportSetting>\n    <InstalledProducts>\n      <Product>\n        <DisplayName>Totally Integrated Automation Portal</DisplayName>\n        <DisplayVersion>V18</DisplayVersion>\n      </Product>\n      <OptionPackage>\n        <DisplayName>TIA Portal Openness</DisplayName>\n        <DisplayVersion>V18</DisplayVersion>\n      </OptionPackage>\n      <OptionPackage>\n        <DisplayName>TIA Portal Version Control Interface</DisplayName>\n        <DisplayVersion>V18</DisplayVersion>\n      </OptionPackage>\n      <Product>\n        <DisplayName>STEP 7 Professional</DisplayName>\n        <DisplayVersion>V18</DisplayVersion>\n      </Product>\n      <OptionPackage>\n        <DisplayName>STEP 7 Safety</DisplayName>\n        <DisplayVersion>V18</DisplayVersion>\n      </OptionPackage>\n      <Product>\n        <Di